In [4]:
import pandas as pd

In [5]:
lexique_df = pd.read_csv("../paraphone/paraphone/data/dictionaries/lexique_383.tsv", delimiter="\t")

In [6]:
short_df = lexique_df[["ortho", "phon", "syll"]]

In [9]:
phonemes = set()
for i, row in short_df.iterrows():
    phonemes.update(set(row["phon"]))
phonemes

{'1',
 '2',
 '5',
 '8',
 '9',
 '@',
 'E',
 'G',
 'N',
 'O',
 'R',
 'S',
 'Z',
 'a',
 'b',
 'd',
 'e',
 'f',
 'g',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '§',
 '°'}

In [10]:
len(lexique_df)

142694

In [13]:
import  re
regex = re.compile("8(.)")
with_8 = set()
for i, row in short_df.iterrows():
    phones: str = row["phon"]
    with_8.update(set(regex.findall(phones)))
with_8

{'2', '5', '9', '@', 'E', 'O', 'a', 'e', 'i', 'o', 'y', '§'}